In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()


In [3]:
vocab = list(set(sentence))
print(vocab)

['is', 'Repeat', 'the', 'memory', 'medicine', 'for', 'best']


In [4]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [5]:
print(word2index)


{'is': 1, 'Repeat': 2, 'the': 3, 'memory': 4, 'medicine': 5, 'for': 6, 'best': 7, '<unk>': 0}


In [6]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'is', 2: 'Repeat', 3: 'the', 4: 'memory', 5: 'medicine', 6: 'for', 7: 'best', 0: '<unk>'}


In [7]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환.
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [8]:
X, Y = build_data(sentence, word2index)


In [9]:
print(X)
print(Y)

tensor([[2, 1, 3, 7, 5, 6]])
tensor([[1, 3, 7, 5, 6, 4]])


In [10]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [11]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [12]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [13]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[-0.1967,  0.2442, -0.1191,  0.1330, -0.2939,  0.1491,  0.2141, -0.0830],
        [-0.1611,  0.3971, -0.0390,  0.0362,  0.0684,  0.1599,  0.2912,  0.1723],
        [-0.1148, -0.0026, -0.0034,  0.0216, -0.0676, -0.0143, -0.0186,  0.1144],
        [ 0.1153,  0.2484, -0.3023,  0.2189, -0.4471, -0.1066,  0.1089,  0.2451],
        [-0.1248,  0.6201, -0.0463,  0.0158,  0.0496,  0.2726,  0.4187,  0.1636],
        [-0.2707,  0.2639, -0.0181,  0.0704,  0.0221,  0.0719,  0.2272,  0.0859]],
       grad_fn=<ViewBackward>)


In [14]:
print(output.shape)


torch.Size([6, 8])


In [15]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [16]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0333 
Repeat is is best is is is

[41/201] 1.4784 
Repeat is the best medicine for memory

[81/201] 0.8678 
Repeat is the best medicine for memory

[121/201] 0.4543 
Repeat is the best medicine for memory

[161/201] 0.2433 
Repeat is the best medicine for memory

[201/201] 0.1447 
Repeat is the best medicine for memory

